# Test Agent Endpoint

This notebook tests both:
1. Direct LangGraph agent invocation
2. Full backend endpoint (with auth + transcription)

In [ ]:
# Install dependencies if needed
# !pip install httpx langgraph-sdk

In [ ]:
import httpx
from langgraph_sdk import get_client
import asyncio
from io import BytesIO

⏺ import httpx
from langgraph_sdk import get_client
from io import BytesIO

# Configuration
LANGGRAPH_URL = "https://noon-2eb9c3ed55f158b6b
1035af721a35d8f.us.langgraph.app"
BASE_URL = "http://localhost:8000"

# Test LangGraph directly
async def test_langgraph(query: str):
    client = get_client(url=LANGGRAPH_URL)
    input_state = {
        "query": query,
        "auth": {"user_id": "test",
"google_tokens": {}},
        "success": False,
        "request": "no-action",
        "metadata": {}
    }
    result = await
client.runs.wait(thread_id=None,
assistant_id="agent", input=input_state)
    print(f"Query: {query}")
    print(f"Request: {result.get('request')},
Success: {result.get('success')}")
    print(f"Metadata:
{result.get('metadata')}\n")
    return result

# Run tests
queries = [
    "What am I doing next weekend?",
    "Show me my team meeting",
    "Create a meeting tomorrow at 2pm"
]

for q in queries:
    print("="*60)
    await test_langgraph(q)

# print("✓ Check traces at:
# https://smith.langchain.com")


## Option 1: Test LangGraph Agent Directly

This bypasses your backend and calls the deployed agent directly.

In [ ]:
# Configuration
LANGGRAPH_URL = "https://noon-2eb9c3ed55f158b6b1035af721a35d8f.us.langgraph.app"

In [ ]:
async def test_langgraph_direct(query: str):
    """Test LangGraph agent directly."""
    print(f"Testing query: {query}\n")
    
    # Create client
    client = get_client(url=LANGGRAPH_URL)
    
    # Input state
    input_state = {
        "query": query,
        "auth": {
            "user_id": "test-user",
            "google_tokens": {
                "access_token": "fake-token",
                "refresh_token": "fake-refresh",
            }
        },
        "success": False,
        "request": "no-action",
        "metadata": {}
    }
    
    # Invoke agent
    result = await client.runs.wait(
        thread_id=None,
        assistant_id="agent",
        input=input_state,
    )
    
    print("✓ Result:")
    print(f"  Success: {result.get('success')}")
    print(f"  Request: {result.get('request')}")
    print(f"  Metadata: {result.get('metadata')}")
    print("\n📊 Check traces at: https://smith.langchain.com")
    
    return result

# Run it
await test_langgraph_direct("What am I doing next weekend?")

In [ ]:
# Try different queries
test_queries = [
    "Show me my team meeting",
    "Create a meeting tomorrow at 2pm",
    "What's on my calendar next Monday?",
    "Cancel my dentist appointment",
]

for query in test_queries:
    print(f"\n{'='*60}")
    result = await test_langgraph_direct(query)
    print(f"{'='*60}")

## Option 2: Test Full Backend Endpoint

This tests the complete flow: Auth → Backend → Transcription → Agent

In [ ]:
# Configuration
BASE_URL = "http://localhost:8000"

# Your phone number for auth
PHONE_NUMBER = "+1234567890"  # Replace with your phone

In [ ]:
# Step 1: Request OTP
async with httpx.AsyncClient() as client:
    response = await client.post(
        f"{BASE_URL}/auth/otp",
        json={"phone": PHONE_NUMBER}
    )
    print(f"OTP sent: {response.status_code}")
    print(response.json())

In [ ]:
# Step 2: Verify OTP (enter the code you received)
OTP_CODE = input("Enter OTP code: ")

async with httpx.AsyncClient() as client:
    response = await client.post(
        f"{BASE_URL}/auth/verify",
        json={"phone": PHONE_NUMBER, "code": OTP_CODE}
    )
    
    if response.status_code == 200:
        auth_data = response.json()
        ACCESS_TOKEN = auth_data["session"]["access_token"]
        print("✓ Authenticated successfully!")
    else:
        print(f"❌ Auth failed: {response.text}")

In [ ]:
# Step 3: Check Google account
async with httpx.AsyncClient() as client:
    response = await client.get(
        f"{BASE_URL}/google-accounts/",
        headers={"Authorization": f"Bearer {ACCESS_TOKEN}"}
    )
    
    if response.status_code == 200:
        accounts = response.json()
        print(f"✓ Found {len(accounts)} Google account(s)")
        for acc in accounts:
            print(f"  - {acc['email']}")
    else:
        print("⚠ No Google accounts linked")
        print("Link one with: POST /google-accounts/oauth/start")

In [ ]:
# Step 4: Call agent endpoint with audio file
async def call_agent_endpoint(audio_file_path: str = None):
    """Call the /agent/action endpoint."""
    
    async with httpx.AsyncClient() as client:
        if audio_file_path:
            # Use real audio file
            with open(audio_file_path, "rb") as f:
                files = {"file": (audio_file_path, f, "audio/wav")}
                response = await client.post(
                    f"{BASE_URL}/agent/action",
                    files=files,
                    headers={"Authorization": f"Bearer {ACCESS_TOKEN}"},
                    timeout=30.0
                )
        else:
            # Use test data (will likely fail transcription)
            print("⚠ Using fake audio data (will fail transcription in production)")
            files = {"file": ("test.wav", BytesIO(b"fake audio"), "audio/wav")}
            response = await client.post(
                f"{BASE_URL}/agent/action",
                files=files,
                headers={"Authorization": f"Bearer {ACCESS_TOKEN}"},
                timeout=30.0
            )
        
        print(f"\nStatus: {response.status_code}")
        result = response.json()
        print(f"Response: {result}")
        
        if response.status_code == 200:
            print("\n✓ Success!")
            print("📊 Check traces at: https://smith.langchain.com")
        
        return result

# Call with real audio file (replace with your file path)
# result = await call_agent_endpoint("/path/to/your/audio.wav")

# Or call with test data
result = await call_agent_endpoint()

## View Traces

After running the agent, view traces at:
**https://smith.langchain.com**

Navigate to:
1. Your project (usually "noon")
2. Click "Traces" in sidebar
3. You'll see the agent execution with:
   - Input query
   - Intent classification
   - Metadata extraction
   - LLM calls
   - Final output